In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def job_finder():
    key = input("Enter your skills or designation: ").strip()
    key_url = key.replace(" ", "%20")

    company_names = []
    skills_list = []
    locations = []
    experiences = []

    for i in range(1, 4):
        url = (
            f'https://www.timesjobs.com/candidate/job-search.html?'
            f'from=submit&luceneResultSize=25&txtKeywords={key_url}'
            f'&postWeek=60&searchType=personalizedSearch&actualTxtKeywords={key_url}'
            f'&searchBy=0&rdoOperator=OR&pDate=I&sequence={i}&startPage=1'
        )

        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.text, "lxml")
        except requests.RequestException as e:
            print(f"Error fetching page {i}: {e}")
            continue

        jobs = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')

        if not jobs:
            print(f"No jobs found on page {i}.")
            continue

        for job in jobs:
            company = job.find('h3', class_='joblist-comp-name')
            comp_text = company.get_text(strip=True) if company else "N/A"
            company_names.append(' '.join(comp_text.split()))

            skills_div = job.find('div', class_='srp-skills')
            if skills_div:
                skills = [span.get_text(strip=True) for span in skills_div.find_all('span', class_=False)]
                skills_text = ', '.join(skills)
            else:
                skills_text = "N/A"
            skills_list.append(skills_text)

            loc_tag = job.find('li', class_='srp-zindex location-tru')
            loc_text = loc_tag.get_text(strip=True) if loc_tag else "N/A"
            locations.append(', '.join(loc_text.split(',')))

            exp_text = "N/A"
            for li in job.find_all('li'):
                if li.find('i', class_='srp-icons experience'):
                    exp_text = li.get_text(strip=True)
                    break
            experiences.append(exp_text)

    result = {
        "Company Name": company_names,
        "Skills": skills_list,
        "Location": locations,
        "Experience": experiences
    }

    print(f"Extracted {len(company_names)} job postings.")
    return result

# Run scraper and save results
data = job_finder()
df = pd.DataFrame(data)
df.to_csv('company.csv', index=False)
print(df.head())


Enter your skills or designation: python developer
Extracted 75 job postings.
       Company Name                                             Skills  \
0             pando  python development, django framework, sql data...   
1             IQVIA  full stack development, python programming, sc...   
2     NuageBiz.Tech  python, mobile, team player, svn, security, gi...   
3  Analytics Vidhya  python, css, html5, javascript, sql, django, l...   
4          zenga tv                  python, django, html5, javascript   

                            Location   Experience  
0                            Chennai  2 - 3 Years  
1              Bengaluru / Bangalore  3 - 6 Years  
2                Noida/Greater Noida  2 - 5 Years  
3  Hyderabad/Secunderabad,   Chennai  2 - 4 Years  
4                            Gurgaon  2 - 5 Years  


In [40]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

def product_finder():
    categories = []
    book_titles = []
    star_ratings = []
    product_price=[]
    in_stock=[]
    base_url = 'https://books.toscrape.com/'
    response = requests.get(base_url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.content, 'lxml')

    books_tag = soup.find('ul', class_='nav nav-list')
    category_list = books_tag.find('ul')
    category_links = category_list.find_all('a')

    for link in category_links:
        category_name = link.get_text(strip=True)
        href = link['href']
        full_url = base_url + href
        slug = href.strip().split('/')[-2]
        categories.append({
            'name': category_name,
            'slug': slug,
            'url': full_url
        })

    key = input("Enter the category name : ").strip()

    matched = None
    for cat in categories:
        if cat['name'].lower() == key.lower():
            matched = cat
            break

    if matched:
        res = requests.get(matched['url'])
        soup = BeautifulSoup(res.content, 'lxml')
        selected_books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')

        #to extract book titles
        for book in selected_books:
            book_tag = book.find('h3')
            book_text = book_tag.get_text(strip=True) if book_tag else "N/A"
            book_titles.append(book_text)

        #print(book_titles)

        #to extract star_ratings
        for book in selected_books:
          star_tag = book.find('p', class_='star-rating')
          if star_tag:
              classes = star_tag.get('class', [])
              rating = classes[1] if len(classes) > 1 else "N/A"
          else:
              rating = "N/A"

          star_ratings.append(rating)

        #print(star_ratings)

        #to extract product_price
        for book in selected_books:
            price_tag=book.find('p',class_='price_color')
            price_text=price_tag.get_text(strip=True) if price_tag else "N/A"
            product_price.append(price_text)

        #print(product_price)

        #to extract stock availability
        for book in selected_books:
            stock_tag=book.find('p',class_='instock availability')
            stock_text=stock_tag.get_text(strip=True) if stock_tag else "N/A"
            in_stock.append(stock_text)

        #print(in_stock)

        result={
            "Book_Title":book_titles,
            "Star_Rating":star_ratings,
            "Product_Price":product_price,
            "In_Stock":in_stock}

        return result


    else:
        print("\n❌ Invalid category. Please choose exactly from the list.")

data=product_finder()
df = pd.DataFrame(data)
df.to_csv('products.csv', index=True)
print(df.head())

Enter the category name : travel
                           Book_Title Star_Rating Product_Price  In_Stock
0             It's Only the Himalayas         Two        £45.17  In stock
1           Full Moon over Noah’s ...        Four        £49.43  In stock
2      See America: A Celebration ...       Three        £48.87  In stock
3  Vagabonding: An Uncommon Guide ...         Two        £36.94  In stock
4                Under the Tuscan Sun       Three        £37.33  In stock


In [63]:
#web_scrapping for "Quotes_to_Scrape" site......
import pandas as pd
import requests
from bs4 import BeautifulSoup

def quote_finder():
    #tag_key=input("Enter the tag name: ").strip()
    base_url = 'https://quotes.toscrape.com/'
    response = requests.get(base_url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.content, 'html.parser')

    tags=[]
    quotes=[]
    authors=[]

    tags_list=soup.find('div',class_='col-md-4 tags-box')
    tag_links=tags_list.find_all('a')
    for link in tag_links:
      tag_name = link.get_text(strip=True)
      href = link['href']
      full_url = base_url + href
      tags.append({
        'name': tag_name,
        'url': full_url})



   #
    quotes_list = soup.find_all('div', class_='quote')

    for quote_div in quotes_list:
        quote_text = quote_div.find('span', class_='text').text
        author=quote_div.find('small',class_='author').text
        quotes.append(quote_text)
        authors.append(author)

    return tags,quotes,authors

quote_finder()




([{'name': 'love', 'url': 'https://quotes.toscrape.com//tag/love/'},
  {'name': 'inspirational',
   'url': 'https://quotes.toscrape.com//tag/inspirational/'},
  {'name': 'life', 'url': 'https://quotes.toscrape.com//tag/life/'},
  {'name': 'humor', 'url': 'https://quotes.toscrape.com//tag/humor/'},
  {'name': 'books', 'url': 'https://quotes.toscrape.com//tag/books/'},
  {'name': 'reading', 'url': 'https://quotes.toscrape.com//tag/reading/'},
  {'name': 'friendship',
   'url': 'https://quotes.toscrape.com//tag/friendship/'},
  {'name': 'friends', 'url': 'https://quotes.toscrape.com//tag/friends/'},
  {'name': 'truth', 'url': 'https://quotes.toscrape.com//tag/truth/'},
  {'name': 'simile', 'url': 'https://quotes.toscrape.com//tag/simile/'}],
 ['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  '“There are only two ways to live you